In [2]:
import numpy as np
import pandas as pd


In [3]:
houseDf = pd.read_csv("Ames_Housing_Sales.csv")
houseDf.shape


(1379, 80)

In [4]:
houseDf.dtypes.value_counts()
# Isto e, podemos ver que nossos regressores contem 43 classificadores categoricos (possivelmente),
#21 continuos e 16 que precisam ser analisados de maneira mais profunda, pois podem ser categoricos,
# intervalares, etc..

object     43
float64    21
int64      16
dtype: int64

In [5]:
houseDf[houseDf.Utilities=='NoSeWa']

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
893,1344.0,0.0,0.0,None,3,1Fam,TA,No,111.0,354.0,...,233.0,Pave,7,819.0,NoSeWa,0.0,1958,1958,2009,137500.0


In [6]:
categorical_columns = houseDf.loc[:,houseDf.dtypes==np.object]
categorical_columns.head(3)
num_ohc_cols = categorical_columns.apply(pd.Series.nunique, axis = 0)
num_ohc_cols.sort_values().sum() # => 258 atributos, porem cada um precisa de -1 uma classe

total_atributos = num_ohc_cols.apply(lambda x: x-1).sort_values().sum()
categorical_columns.head(3)


,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,MiscFeature,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,Utilities
0,None,1Fam,TA,No,GLQ,Unf,Gd,Y,Norm,Norm,...,None,CollgCr,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
1,None,1Fam,TA,Gd,ALQ,Unf,Gd,Y,Feedr,Norm,...,None,Veenker,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub
2,None,1Fam,TA,Mn,GLQ,Unf,Gd,Y,Norm,Norm,...,None,CollgCr,Y,None,CompShg,Gable,Normal,WD,Pave,AllPub


In [7]:
copyDf = houseDf.copy()

In [8]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
alley = num_ohc_cols.index[0]

le = LabelEncoder()
ohc = OneHotEncoder()

dat = le.fit(copyDf['Utilities'])
ndat = le.transform(copyDf['Utilities'])
dat.inverse_transform([1,1,0])

array(['NoSeWa', 'NoSeWa', 'AllPub'], dtype=object)

In [9]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

data_ohc = houseDf.copy()

le = LabelEncoder()
ohc = OneHotEncoder()

for col in num_ohc_cols.index:
    dat = le.fit_transform(copyDf[col]).astype(np.int).reshape(-1,1)
    data_ohc = data_ohc.drop(col, axis=1)
    
    new_dat = ohc.fit(dat)
    new_dat = ohc.transform(dat).toarray()
    
    n_cols = new_dat.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]
    
    new_df = pd.DataFrame(new_dat, columns = col_names, index = data_ohc.index)
    
    data_ohc = pd.concat([data_ohc, new_df], axis=1)



In [10]:
nova_menos_original = data_ohc.shape[1]-houseDf.shape[1]
nova_menos_original

215

In [11]:
houseDf.shape
# Removendo os dados categoricos do dataframe original
house_nonNum = houseDf.select_dtypes(exclude=['object'])
house_nonNum.shape


(1379, 37)

In [12]:
# Exercicio 4:

from sklearn.model_selection import train_test_split

y_col = 'SalePrice'
feature_cols = [x for x in house_nonNum.columns if x!=y_col]

X_data = house_nonNum[feature_cols]
y_data = house_nonNum[y_col]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3,random_state=42)

feature_cols_ohc = [x for x in data_ohc.columns if x!=y_col]
X_data_ohc = data_ohc[feature_cols_ohc]
y_data_ohc = data_ohc[y_col]

X_train_ohc, X_test_ohc, y_train_ohc, y_test_ohc = train_test_split(X_data_ohc, y_data_ohc, test_size= 0.3, random_state=42)

X_train.head(3)

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,OverallCond,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
461,630.0,0.0,0.0,1,515.0,0.0,1,0,115.0,0.0,...,8,4,0.0,0.0,3,630.0,0.0,1970,2002,2009
976,845.0,0.0,0.0,3,0.0,0.0,0,0,0.0,0.0,...,3,4,0.0,0.0,5,0.0,186.0,1957,1957,2009
1128,728.0,728.0,0.0,3,0.0,0.0,0,0,728.0,0.0,...,5,6,0.0,0.0,8,728.0,100.0,2005,2005,2008


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

LR = LinearRegression()

error_df = list()

LR = LR.fit(X_train, y_train) # Treino meu modelo, isto e, calculo meus parametros com a base de treino

y_train_pred = LR.predict(X_train) # Calculo os valores de Y, a partir do meu modelo de treino
y_test_pred = LR.predict(X_test) # Calculo o mesmo para a base de teste

error_df.append(pd.Series({'train': mean_squared_error(y_train, y_train_pred),
                           'test' : mean_squared_error(y_test,  y_test_pred)},
                           name='no enc'))

In [14]:
# Agora para os dados em data_ohc

LR_ohc = LinearRegression()

model = LR_ohc.fit(X_train_ohc, y_train_ohc)
y_train_pred_ohc = model.predict(X_train_ohc)
y_test_pred_oh = model.predict(X_test_ohc)

error_df.append(pd.Series({'train': mean_squared_error(y_train_ohc, y_train_pred_ohc),
                           'test' : mean_squared_error(y_test_ohc,  y_test_pred)},
                          name='one-hot enc'))

In [15]:
error_df = pd.concat(error_df, axis=1)

In [16]:
error_df # Error no one-hot encode para treino e teste sao diferentes, pois atributos categoricos geram
# overfit na base de dados

,no enc,one-hot enc
test,1.372182e+09,1.372182e+09
train,1.131507e+09,3.177310e+08


In [400]:
# Iremos normalizar nossa base de dados para calculo do erro
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

scalers = {'standard': StandardScaler(),
           'minmax': MinMaxScaler(),
           'maxabs': MaxAbsScaler()}

training_test_sets = {
    'not_encoded': (X_train, y_train, X_test, y_test),
    'one_hot_encoded': (X_train_ohc, y_train_ohc, X_test_ohc, y_test_ohc)}

mask = X_train.dtypes == np.float
float_columns = X_train.columns[mask]


LR = LinearRegression()

errors = {}
for encoding_label, (_X_train, _y_train, _X_test, _y_test) in training_test_sets.items():
    for scaler_label, scaler in scalers.items():
        trainingset = _X_train.copy()  # vamos fazer uma cópia para não bagunçar o original
        testset = _X_test.copy()
        trainingset[float_columns] = scaler.fit_transform(trainingset[float_columns])
        testset[float_columns] = scaler.transform(testset[float_columns])
        # Utilizamos somente o transform para o testset, pois nao queremos calcular a media,
        # e o desvio padrao a partir da base de test, o que poderia mudar nossa base de test significativamente
        # portanto, utilizaremos o mesmo padrao (media, e desvio) da base de treino
        
        LR.fit(trainingset, _y_train)
        predictions = LR.predict(testset)
        key = encoding_label + ' - ' + scaler_label + ' scaling'
        errors[key] = mean_squared_error(_y_test, predictions)
        
errors = pd.Series(errors)
print(errors.to_string())
print('-' * 80)
for key, error_val in errors.items():
    print(key, error_val)

not_encoded - maxabs scaling          1.372024e+09
not_encoded - minmax scaling          1.372329e+09
not_encoded - standard scaling        1.372182e+09
one_hot_encoded - maxabs scaling      8.065328e+09
one_hot_encoded - minmax scaling      8.065328e+09
one_hot_encoded - standard scaling    8.065328e+09
--------------------------------------------------------------------------------
not_encoded - maxabs scaling 1372023689.44
not_encoded - minmax scaling 1372329164.98
not_encoded - standard scaling 1372182358.93
one_hot_encoded - maxabs scaling 8065327607.09
one_hot_encoded - minmax scaling 8065327607.27
one_hot_encoded - standard scaling 8065327607.25


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

lr = LinearRegression()

standard = StandardScaler()
training = X_train.copy()
test = X_test.copy()

training[float_columns]